In [1]:
import tensorflow as tf
import csv

with open("train.csv",'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
print ("   ")
with open("eval.csv",'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)

['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
['0', 'male', '22', '1', '0', '7.25', 'Third', 'unknown', 'Southampton', '0']
['1', 'female', '38', '1', '0', '71.2833', 'First', 'C', 'Cherbourg', '0']
['1', 'female', '26', '0', '0', '7.925', 'Third', 'unknown', 'Southampton', '1']
['1', 'female', '35', '1', '0', '53.1', 'First', 'C', 'Southampton', '0']
['0', 'male', '28', '0', '0', '8.4583', 'Third', 'unknown', 'Queenstown', '1']
['0', 'male', '2', '3', '1', '21.075', 'Third', 'unknown', 'Southampton', '0']
['1', 'female', '27', '0', '2', '11.1333', 'Third', 'unknown', 'Southampton', '0']
['1', 'female', '14', '1', '0', '30.0708', 'Second', 'unknown', 'Cherbourg', '0']
['1', 'female', '4', '1', '1', '16.7', 'Third', 'G', 'Southampton', '0']
['0', 'male', '20', '0', '0', '8.05', 'Third', 'unknown', 'Southampton', '1']
['0', 'male', '39', '1', '5', '31.275', 'Third', 'unknown', 'Southampton', '0']
['0', 'female', '14', '0', '0',

In [2]:
train_file_path = "train.csv"
test_file_path = "eval.csv"

In [3]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22,1,0,7.25,Third,unknown,Southampton,0
1,female,38,1,0,71.2833,First,C,Cherbourg,0
1,female,26,0,0,7.925,Third,unknown,Southampton,1
1,female,35,1,0,53.1,First,C,Southampton,0
0,male,28,0,0,8.4583,Third,unknown,Queenstown,1
0,male,2,3,1,21.075,Third,unknown,Southampton,0
1,female,27,0,2,11.1333,Third,unknown,Southampton,0
1,female,14,1,0,30.0708,Second,unknown,Cherbourg,0
1,female,4,1,1,16.7,Third,G,Southampton,0


In [5]:
LABEL_COLUMN = 'alone'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [7]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [8]:
show_batch(raw_train_data)

survived            : [1 0 1 0 1]
sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [28. 36. 33. 16. 25.]
n_siblings_spouses  : [0 1 1 1 1]
parch               : [0 0 0 3 1]
fare                : [56.496 78.85  53.1   34.375 30.   ]
class               : [b'Third' b'First' b'First' b'Third' b'Second']
deck                : [b'unknown' b'C' b'E' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']


In [9]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

survived            : [0 1 0 1 0]
sex                 : [b'male' b'male' b'female' b'female' b'male']
age                 : [29. 48. 28. 28. 47.]
n_siblings_spouses  : [0 1 0 1 0]
parch               : [0 0 2 0 0]
fare                : [30.    76.729  7.75  16.1    7.25 ]
class               : [b'First' b'First' b'Third' b'Third' b'Third']
deck                : [b'D' b'D' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Queenstown' b'Southampton' b'Southampton']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

survived            : [0 0 0 0 1]
age                 : [30. 24. 28. 41. 24.]
n_siblings_spouses  : [0 2 0 0 1]
class               : [b'Second' b'Second' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare','alone']
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0 , 0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

survived            : [0. 0. 1. 0. 0.]
age                 : [18. 24. 60. 30. 32.]
n_siblings_spouses  : [0. 0. 1. 0. 0.]
parch               : [0. 0. 1. 0. 0.]
fare                : [11.5   79.2   79.2    8.663  7.896]


In [12]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [13]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [14]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 0.    36.     1.     0.    78.85 ]
 [ 0.     2.     0.     1.    10.462]
 [ 0.    25.     0.     0.     7.742]
 [ 0.    24.     0.     0.    13.   ]
 [ 1.    26.     0.     0.     7.925]]

[0 0 1 1 1]


In [15]:
show_batch(raw_train_data)

survived            : [0 1 0 0 0]
sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [66. 28. 22. 28. 57.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 10.5     7.787   7.229 227.525  10.5  ]
class               : [b'Second' b'Third' b'Third' b'First' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'E']
embark_town         : [b'Southampton' b'Queenstown' b'Cherbourg' b'Cherbourg' b'Southampton']


In [16]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [17]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [18]:
NUMERIC_FEATURES = ['survived','age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [19]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg' b'Southampton']
numeric             : [[ 1.     1.     1.     2.    20.575]
 [ 1.    63.     0.     0.     9.587]
 [ 1.    28.     0.     0.     7.879]
 [ 0.    34.5    0.     0.     6.438]
 [ 0.    28.     0.     0.     7.896]]


In [20]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [21]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,survived,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.387560,29.631308,0.545455,0.379585,34.385399
std,0.487582,12.511818,1.151090,0.792999,54.597730
min,0.000000,0.750000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,0.000000,7.895800
50%,0.000000,28.000000,0.000000,0.000000,15.045800
75%,1.000000,35.000000,1.000000,0.000000,31.387500
max,1.000000,80.000000,8.000000,5.000000,512.329200


In [22]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [23]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


In [24]:
import functools

normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(5,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f26cc60a290>, mean=array([ 0.388, 29.631,  0.545,  0.38 , 34.385]), std=array([ 0.488, 12.512,  1.151,  0.793, 54.598])))

In [25]:
example_batch['numeric']

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[ 1.   , 21.   ,  0.   ,  0.   ,  7.796],
       [ 1.   , 24.   ,  0.   ,  0.   , 69.3  ],
       [ 0.   , 28.   ,  0.   ,  0.   ,  8.05 ],
       [ 0.   , 32.   ,  0.   ,  0.   ,  7.925],
       [ 1.   , 28.   ,  0.   ,  0.   , 13.   ]], dtype=float32)>

In [26]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.256, -0.69 , -0.474, -0.479, -0.487],
       [ 1.256, -0.45 , -0.474, -0.479,  0.639],
       [-0.795, -0.13 , -0.474, -0.479, -0.482],
       [-0.795,  0.189, -0.474, -0.479, -0.485],
       [ 1.256, -0.13 , -0.474, -0.479, -0.392]], dtype=float32)

In [28]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown']
}


In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [32]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [33]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     0.     1.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     1.256 -0.69  -0.474 -0.479
 -0.487  1.     0.   ]


In [34]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [35]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [36]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 3s 25ms/step - loss: 0.2842 - accuracy: 0.8740
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.0219 - accuracy: 0.9984
Epoch 3/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 4/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 9.7173e-04 - accuracy: 1.0000
Epoch 6/20
126/126 [==============================] - 1s 4ms/step - loss: 6.0358e-04 - accuracy: 1.0000
Epoch 7/20
126/126 [==============================] - 1s 4ms/step - loss: 4.1137e-04 - accuracy: 1.0000
Epoch 8/20
126/126 [==============================] - 1s 4ms/step - loss: 2.9414e-04 - accuracy: 1.0000
Epoch 9/20
126/126 [==============================] - 1s 4ms/step - loss: 2.1750e-04 - accuracy: 1.0000
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 1.63

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 1s 11ms/step - loss: 2.2010e-05 - accuracy: 1.0000

Test Loss 2.201018113595946e-05, Test Accuracy 1.0


In [39]:
predictions = model.predict(test_data)

# Show some results
for prediction, alone in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted matching: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("ALONE" if bool(alone) else "FAMILY"))

Predicted matching: 0.00%  | Actual outcome:  FAMILY
Predicted matching: 100.00%  | Actual outcome:  ALONE
Predicted matching: 100.00%  | Actual outcome:  ALONE
Predicted matching: 100.00%  | Actual outcome:  ALONE
Predicted matching: 100.00%  | Actual outcome:  ALONE
